<img style="float:left;margin-right:1em;width:5em;" src="img/smelli.svg">

# `smelli` - the **SME**FT **L**ike**LI**hood

<div style="clear:left;"></div>

Lecture by Peter Stangl

- `smelli` articles: [arXiv:1810.07698](https://arxiv.org/abs/1810.07698), [arXiv:2012.12211](https://arxiv.org/abs/2012.12211)

- `smelli` API documentation: https://smelli.github.io/smelli/

- `smelli` source: https://github.com/smelli/smelli

- Slides of this lecture: https://github.com/peterstangl/flavio-lecture/tree/Ljubljana

# The global SMEFT likelihood

## Implementation the global SMEFT likelihood

-  Based on
<img style="display:inline;margin-left:0.1em;margin-right:0.3em;margin-bottom:0.55em;height:.9em;" src="img/flavio.svg">**flavio**,
<img style="display:inline;margin-left:0.1em;margin-right:0.3em;margin-bottom:0.55em;height:.9em;" src="img/wilson.png">**wilson**, and
<img style="display:inline;margin-left:0.1em;margin-right:0.3em;margin-bottom:0.55em;height:.9em;" src="img/wcxf.svg">**WCxf**,
we have started building the <br> global **SME**FT **L**ike**LI**hood <img style="display:inline;margin-left:0.1em;margin-right:0.3em;margin-bottom:0.55em;height:.9em;" src="img/smelli.svg">`smelli` https://github.com/smelli
  <p style="float:left;width:100%;margin:0em;text-align:right;font-size:60%;color:#91ac6b;padding-right:3em;padding-left:3em;margin-top:-3em;">Aebischer, Kumar, PS, Straub, arXiv:1810.07698
    <br>
    PS, arXiv:2012.12211
</p>
  <p style="clear:left;margin:0em;height:0em;padding:0em"></p>

<ul style="float:left;width:26em;margin-top:0.2em;"><li>
    $
  L(\vec C) \approx
  \prod_{i} L_\text{exp}^i(\vec{O}_\text{th}(\vec C, \vec \theta_0))
  \times
  \tilde{L}_\text{exp}(\vec{O}_\text{th}(\vec C, \vec \theta_0))
     $
    where
    <ul>
        <li>$\vec C$ WET or SMEFT Wilson coefficients</li>
        <li>$\vec \theta_0$ fixed nuisance parameters</li>
        <li>$\vec{O}_\text{th}(\vec C, \vec \theta_0)$ observable predictions</li>
        <li>$ L_\text{exp}^i(\vec{O})$ experimental likelihood from<br>measurement $i$ for observables $\vec{O}$</li>
        <li>$\tilde{L}_\text{exp}(\vec O)$ modified exp. likelihood:<br>$-2\ln \tilde{L}_\text{exp}(\vec O)=\vec D^T (\Sigma_\text{exp}+\Sigma_\text{th})^{-1} \vec D \,,$<br>with $\vec D = \vec O - \vec O_\text{exp}$ and
      covariance<br>matrices $\Sigma_\text{exp,th}$ (Gaussian approx.)</li>
    </ul>
    </li></ul>
        <img style="float:left;margin-left:2em;margin-top:1em;height:12em;"; src="img/sketch-new.svg"/>

# Using `smelli`

## Installing `smelli`

- Requirements:
    - working installation of **Python** version **3.7** or above
    - **pip** (Python package manager)

- Installation from the command line:

  **`python3 -m pip install smelli --user`**

  - downloads `smelli` with all dependencies from Python package archive (PyPI)
  - installs it in user's home directory (no need to be root)

## Instantiating the likelihood

In [ ]:
from smelli import GlobalLikelihood

In [ ]:
gl = GlobalLikelihood()

In [ ]:
gl.eft, gl.basis

In [ ]:
gl_wet = GlobalLikelihood(eft='WET', basis='flavio')
gl_wet.eft, gl_wet.basis

## Fixing a point in WC space: 3 equivalent ways

In [ ]:
pp = gl.parameter_point({'lq3_2223': 1e-9}, scale=1000)

In [ ]:
from wilson import Wilson
w = Wilson({'lq3_2223': 1e-9}, scale=1000, eft='SMEFT', basis='Warsaw')
pp = gl.parameter_point(w)

In [ ]:
! cat my_wcxf.yaml

In [ ]:
pp = gl.parameter_point('my_wcxf.yaml')

## Computing the likelihood

In [ ]:
%%time
pp.log_likelihood_dict()

In [ ]:
%%time
pp.log_likelihood_global()

In [ ]:
pp.chi2_dict()

In [ ]:
pp.pvalue_dict()

## Table of observables

In [ ]:
df = pp.obstable()

In [ ]:
df

In [ ]:
df.sort_values('pull SM')[:10]

In [ ]:
df.loc[['Rtaul(B->D*lnu)']]

## Plots

In [ ]:
import flavio.plots as fpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
import warnings
warnings.simplefilter('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
rcParams.update({'font.size': 18, 'xtick.labelsize': 18, 'ytick.labelsize': 18, 'axes.labelsize':18})

In [ ]:
gl_ewpt_higgs = GlobalLikelihood(fix_ckm=True, include_likelihoods=['likelihood_ewpt.yaml', 'likelihood_higgs.yaml'])

In [ ]:
def wc_fct(S, T):
    return {
        'phiWB': S * 7.643950529889027e-08, # gL*gY/(16*pi*v**2)
        'phiD': -T * 2.5793722852276787e-07, # gL**2*gY**2/(2*pi*v**2)/(gL**2+gY**2)
    }
scale = 91.1876

In [ ]:
%%time

plot_data = gl_ewpt_higgs.plot_data_2d(
    wc_fct, scale,
    -0.15, 0.25, -0.1, 0.3,
    steps=10,
    threads=8,
)

In [ ]:
%%time

chi2_min = gl_ewpt_higgs.chi2_min(wc_fct, scale, plotdata=plot_data)

In [ ]:
plotdata = {k:{**v, **chi2_min[k]} for k,v in plot_data.items()}

In [ ]:
levels_1sig = [fpl.delta_chi2(1, dof=2)]
levels_123sig = [fpl.delta_chi2(n_sigma, dof=2) for n_sigma in (1,2,3)]
interpolation_factor = 3

In [ ]:
plt.figure(figsize=(6,6))
fpl.contour(**plotdata['likelihood_higgs.yaml'], levels=levels_1sig, label=r"Higgs ($1\sigma$)",
            interpolation_factor=interpolation_factor, color='C0')
fpl.contour(**plotdata['likelihood_ewpt.yaml'], levels=levels_1sig, label=r"EWPT ($1\sigma$)",
            interpolation_factor=interpolation_factor, color='C1')
fpl.contour(**plotdata['global'], levels=levels_123sig, label=r"global",
            interpolation_factor=interpolation_factor, color='C3')
plt.axhline(c='0.6', linewidth=1)
plt.axvline(c='0.6', linewidth=1)
plt.xlabel(r'$S$')
plt.ylabel(r'$T$')
plt.legend()
plt.show()

## Outlook to `smelli` v3.0 (work in progress)

- Major speed improvement (orders of magnitude)
- High-mass Drell-Yan tails: $pp\to e\nu,\mu\nu$ and $pp\to e^+e^-,\mu^+\mu^-$ (already available in `flavio`)
- EDMs: neutron, atomic, and molecular (already available in `flavio`)

## `flavio` and `smelli` are open source projects

Contributions from the user community are very welcome and encouraged!

If you want to use `flavio` or `smelli` for your research and you
- implement a new observable
- add or update measurements
- have an idea for a new feature

please get in contact by opening an issue on GitHub:

- https://github.com/flav-io/flavio/
- https://github.com/smelli/smelli